In [1]:
from openbb_terminal.sdk import openbb
import yaml

with open('credentials/api.yaml', 'r') as file:
    config = yaml.safe_load(file)

openbb.keys.fmp(key = config['keys']['fmp'], persist = True)



'Defined, test passed'

In [16]:
from urllib.request import urlopen
from urllib.error import HTTPError
import json
import pandas as pd

def available_companies(api_key):
  
    try:
        response = urlopen(f"https://financialmodelingprep.com/api/v3/stock/list?apikey={api_key}")
        data = json.loads(response.read().decode("utf-8"))
    except HTTPError:
        raise ValueError("This endpoint is only for premium members. Please visit the subscription page to upgrade the "
                         "plan (Starter or higher) at https://financialmodelingprep.com/developer/docs/pricing")

    if 'Error Message' in data:
        raise ValueError(data['Error Message'])

    df = pd.DataFrame(data)
    df.loc[df["name"].isna(), "name"] = df["symbol"]
    df = df.set_index("symbol")
    
    df.reset_index(inplace=True)

    return df

In [17]:
companies = available_companies(config['keys']['fmp'])

In [18]:
companies

,symbol,name,price,exchange,exchangeShortName,type
0,PMGOLD.AX,Perth Mint Gold,17.9400,Australian Securities Exchange,ASX,etf
1,FEMACX,The First Trust Combined Series 447: Investmen...,961.5100,Nasdaq,NASDAQ,trust
2,YAFFX,AMG Yacktman Focused Fund,19.8500,Nasdaq,NASDAQ,trust
3,YAFIX,AMG Funds - AMG Yacktman Focused Fund,19.7900,Nasdaq,NASDAQ,trust
4,ZSCCX,Zacks Small-Cap Core Fund,33.7800,Nasdaq,NASDAQ,trust
...,...,...,...,...,...,...
72618,PAQCW,Provident Acquisition Corp.,0.3627,NASDAQ Capital Market,NASDAQ,stock
72619,LLL,JX Luxventure Limited,0.8391,NASDAQ Capital Market,NASDAQ,stock
72620,TPGY-UN,TPG Pace Beneficial Finance Corp.,10.0500,New York Stock Exchange,NYSE,stock
72621,ESP.AS,Esperite N.V.,0.0092,Amsterdam,AMS,stock


In [19]:
companies_nasdaq = companies[(companies['exchangeShortName'] == "NASDAQ") & (companies['type'] == "stock")]
list_ticker = companies_nasdaq.symbol.to_list()

In [21]:
len(list_ticker)

6085

In [29]:
companies_nasdaq.sort_values(by=['symbol'], ascending=False)

,symbol,name,price,exchange,exchangeShortName,type
9628,ZYXI,"Zynex, Inc.",8.3300,NASDAQ Global Select,NASDAQ,stock
10381,ZYNE,"Zynerba Pharmaceuticals, Inc.",0.3530,NASDAQ Global Market,NASDAQ,stock
67733,ZY,Zymergen Inc.,2.4300,NASDAQ Global Select,NASDAQ,stock
27957,ZXZZT,,17.1200,NASDAQ Global Market,NASDAQ,stock
72349,ZXYZ-A,,5.5000,NASDAQ Global Market,NASDAQ,stock
...,...,...,...,...,...,...
60835,AACQU,Artius Acquisition Inc.,11.0500,Nasdaq Global Select,NASDAQ,stock
14836,AACIW,Armada Acquisition Corp. I,0.0601,NASDAQ Global Market,NASDAQ,stock
56631,AACIU,Armada Acquisition Corp. I,10.5300,NASDAQ Global Market,NASDAQ,stock
18371,AACI,Armada Acquisition Corp. I,10.5000,NASDAQ Global Market,NASDAQ,stock


In [37]:
companies_nasdaq[companies_nasdaq['name'] == "Alphabet Inc."]

,symbol,name,price,exchange,exchangeShortName,type
6150,GOOG,Alphabet Inc.,128.54,NASDAQ Global Select,NASDAQ,stock
6611,GOOGL,Alphabet Inc.,128.11,NASDAQ Global Select,NASDAQ,stock
